# Labelled Vector Quantaizer (LVQ) from scrach
## Part One
Remember LVQ algorithms? Well in this problem we would like to implement two LVQ algorithms fora binary classification task.

* 1.  LVQ 1

The dataset to evaluate your code is attached.  Please note that you should split the data into trainingand validation sets (9.0 ratio).  The last column is a label for each point.

Please be advised, any use of python libraries, which have predefined algorithms is strictly prohibited.You should use numpy for your implementation (Tensorflow implementation has bonus score).


In [328]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


dataset = pd.read_csv(r"./data set.txt", header=None)



X = np.array([dataset[0],dataset[1],dataset[2],dataset[3]]).transpose()
Y = np.array(dataset[4])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

dataset.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


### Implementation description
LVQ has six main steps as belows:
1. Initialize weghts
2. Repeat stages 2 to 6 while stop condition is not true
3. For each input sample find nearest prototype vector using some destance measure as vector $W_j$
4. Update $W_j$ like:
    * if label = $W_j$ label then: $W_j = W_j + \alpha \left( t \right) \times (X - W_j)$
    * if label != $W_j$ label then: $W_j = W_j - \alpha \left( t \right) \times (X - W_j)$
5. Reduce learning rate ($\alpha$)
6. check stop condition
    * Stop condition could be number of times that we give input data to the network

As observed in the steps, we need a distance measure to find the nearest prototype vector to the input sample. For simplicity, we could choose *euclidean distance* as a distance measure for our implementation. 

In [329]:
def euclideanDistance(c,x):
    return sqrt(np.dot((c-x),(c-x)))

The next thing needed for the LVQ algorithm, a number of prototype vectors for the problem. Choosing prototype vectors could be a challenging choice as they affect our overall performance and final accuracy score. By choosing these vectors with prior knowledge of the training data, we could achieve better results instead of choosing blindly.
 
As it is a homework and again for fewer context length, we assume two prototype vectors for *negative* and *positive* classes, respectively.  

In [337]:
w = np.random.rand(2,4)*X_train.max()
y_w = np.array([0,1])
print(w)

[[ 5.19548133  4.08533252 12.53875772  1.40327676]
 [ 4.97449953  2.79617278  8.93597108 13.90703441]]


Now we have all the requirements, let's write the LVQ class that uses our *dataset* and prototype vector to fit the model with the data.

In [338]:
class LVQ:
    
    def __init__(self, w, w_label):
        if(w.shape[0] != w_label.shape[0]):
            raise ValueError("Vectors and labels dont't have the same length.")
        self.w = np.array(w)
        self.w_label = np.array(w_label)
        
    def __euclideanDistance(self, c,x):
        return sqrt(np.dot((c-x),(c-x)))
    
    def __findWinner(self, x):
        winnerIdx = 0
        nearestDistance = 1000000
        for i in range(w.shape[0]):
            distance = self.__euclideanDistance(x, w[i])
            if(distance < nearestDistance):
                nearestDistance = distance
                winnerIdx = i
        return winnerIdx
    
    def __update(self, x, x_label, winnerIdx):
        
        delta = (x - self.w[winnerIdx])
        if(x_label == self.w_label[winnerIdx]):
            self.w[winnerIdx] = self.w[winnerIdx] + (delta*self.alpha)
        else:
            self.w[winnerIdx] = self.w[winnerIdx] - (delta*self.alpha)


    def fit(self, X, Y, alpha=0.5, epoch=10):
        self.alpha = alpha 
        if(X.shape[0] != Y.shape[0]):
            raise ValueError("Vectors and labels dont't have the same length.")
        for e in tqdm(range(epoch)):
            self.alpha = alpha*(1-e/epoch)
            l = np.arange(X.shape[0])
            np.random.shuffle(l)            
            for i in l:
                x = X[i]
                y = Y[i]
                winnerIdx = self.__findWinner(x)
                self.__update(x, y, winnerIdx)
                
    def predict(self, X):
        Y_predict = []
        for x in X:
            winnerIdx = self.__findWinner(x)
            Y_predict += [self.w_label[winnerIdx]]
        return np.array(Y_predict)

In [339]:
lvq = LVQ(w, y_w)
lvq.fit(X_train, Y_train, alpha=1, epoch=10)
Y_pred = lvq.predict(X_test)
print(accuracy_score(Y_test, Y_pred))


0.5579710144927537


By initializing prototype vectors having prior knowledge about the data, we could achieve better results. To test this fact, let's consider prototype vectors as a negative sample and a positive sample from our dataset:

In [340]:
pIdx = 0 # Posetive sample index
nIdx = 0 # Negative sample index

for i in range(Y_train.shape[0]):
    if(Y_train[i] == 0):
        nIdx = i
        break
for i in range(Y_train.shape[0]):
    if(Y_train[i] == 1):
        pIdx = i
        break
        
w = np.array([X_train[nIdx],X_train[pIdx]])
y_w = np.array([0,1])

lvq = LVQ(w, y_w)
lvq.fit(X_train, Y_train, alpha=1, epoch=10)
Y_pred = lvq.predict(X_test)
print(accuracy_score(Y_test, Y_pred))


0.6811594202898551


As observed, by choosing prototype vectors as the first and sample of each class in our training set, the accuracy increses. for better results, we could choose much better samples with some preprocessing on our data.